# Membrane Segmentation
This notebook demonstrates how to segment cell membranes from fluorescence microscopy images.

## Import required libraries

In [1]:
import numpy as np
from skimage import io, filters, segmentation
from skimage.morphology import remove_small_objects
import matplotlib.pyplot as plt

## Create synthetic test image

In [2]:
# Create a synthetic membrane-like test image
image = np.zeros((100, 100))
for i in range(5):
    for j in range(5):
        center_x = 10 + i * 20
        center_y = 10 + j * 20
        x, y = np.ogrid[-center_x:100-center_x, -center_y:100-center_y]
        mask = x*x + y*y <= 8*8
        image[mask] = 1

plt.imshow(image, cmap='gray')
plt.axis('off')
plt.savefig('original.png')
plt.close()

## Preprocess image using Gaussian smoothing

In [3]:
smoothed = filters.gaussian(image, sigma=1)
plt.imshow(smoothed, cmap='gray')
plt.axis('off')
plt.savefig('smoothed.png')
plt.close()

## Threshold the image to separate membranes

In [4]:
threshold = filters.threshold_otsu(smoothed)
binary = smoothed > threshold
plt.imshow(binary, cmap='gray')
plt.axis('off')
plt.savefig('binary.png')
plt.close()

## Segment cells using watershed

In [5]:
distance = segmentation.watershed(-smoothed, markers=100, mask=~binary)
labels = remove_small_objects(distance > 0, min_size=100)

plt.imshow(labels, cmap='nipy_spectral')
plt.axis('off')
plt.savefig('segmented.png')
plt.close()

io.imsave('labels.png', (labels * 255 / labels.max()).astype(np.uint8))